In [1]:
import pandas as pd
import numpy as np

## EPH survey data

In [2]:
df = pd.read_csv('./../resumen_EPH_hogar.csv')
df = df.loc[df.IV1 != 9]
df['IV10'] = df['IV10'].map({1: 1, 2: 2, 3: 2, 0: 0, 9: 9})
df['II9'] = df['II9'].map({1: 1, 2: 2, 3: 2, 4: 4, 0: 0})
df['II7'] = df['II7'].map({1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 6, 8: 6, 9: 6, 0: 0})
df['IX_TOT'] = df['IX_TOT'].clip(0, 8)


In [3]:
# for col in df.iloc[:, 2:].columns:
#     print col
#     print sorted(df[col].unique())

## 2010 ARG Census data

In [4]:
path = './../../Datos_censo/Preguntas/'

VIVIENDA_V01 = pd.read_csv(path + 'VIVIENDA-V01.csv', encoding = 'latin-1')
HOGAR_H05 = pd.read_csv(path + 'HOGAR-H05.csv', encoding = 'latin-1') # Pisos
HOGAR_H06 = pd.read_csv(path + 'HOGAR-H06.csv', encoding = 'latin-1') # Cubierta techos
HOGAR_H07 = pd.read_csv(path + 'HOGAR-H07.csv', encoding = 'latin-1') # Cielorraso
HOGAR_H08 = pd.read_csv(path + 'HOGAR-H08.csv', encoding = 'latin-1') # Agua
HOGAR_H09 = pd.read_csv(path + 'HOGAR-H09.csv', encoding = 'latin-1') # Procedencia agua
HOGAR_H10 = pd.read_csv(path + 'HOGAR-H10.csv', encoding = 'latin-1') # Tiene baño
HOGAR_H11 = pd.read_csv(path + 'HOGAR-H11.csv', encoding = 'latin-1') # Baño con cadena
HOGAR_H12 = pd.read_csv(path + 'HOGAR-H12.csv', encoding = 'latin-1') # Desague baño
HOGAR_H13 = pd.read_csv(path + 'HOGAR-H13.csv', encoding = 'latin-1') # Baño uso exclusivo
HOGAR_H14 = pd.read_csv(path + 'HOGAR-H14.csv', encoding = 'latin-1') # Combustible cocina
HOGAR_PROP = pd.read_csv(path + 'HOGAR-PROP.csv', encoding = 'latin-1') # Regimen tenencia
HOGAR_TOTPERS = pd.read_csv(path + 'HOGAR-TOTPERS.csv', encoding = 'latin-1') # Personas en hogar

df_list = [VIVIENDA_V01, HOGAR_H05, HOGAR_H06, HOGAR_H07, HOGAR_H08, HOGAR_H09, 
          HOGAR_H10, HOGAR_H11, HOGAR_H12, HOGAR_H13, HOGAR_H14, HOGAR_PROP, HOGAR_TOTPERS]

## Harmonise variables (let valid answers match exactly in both datasets)

In [5]:

VIVIENDA_V01['2 Departamento'] = VIVIENDA_V01['4 Departamento']
VIVIENDA_V01['3 Pieza de inquilinato'] = VIVIENDA_V01['5 Pieza en inquilinato']
VIVIENDA_V01[u'4 Pieza en hotel / pensión'] = VIVIENDA_V01[u'6 Pieza en hotel familiar o pensión']
VIVIENDA_V01[u'5 Local no construido para habitación'] = VIVIENDA_V01[u'7 Local no construido para habitación']
VIVIENDA_V01['6 Otros'] = VIVIENDA_V01['2 Rancho'] + VIVIENDA_V01['3 Casilla'] + VIVIENDA_V01[u'8 Vivienda móvil'] + VIVIENDA_V01['9 Persona/s viviendo en la calle']

IV1_censo = VIVIENDA_V01[['radio', '1 Casa', '2 Departamento', '3 Pieza de inquilinato', u'4 Pieza en hotel / pensión',
                         u'5 Local no construido para habitación', '6 Otros', 'TOTAL']]
IV3_censo = HOGAR_H05
IV3_censo.columns = ['radio', '1 Mosaico / baldosa / madera / cerámica / alfombra', 
                    '2 Cemento / ladrillo fijo', '3 Ladrillo suelto / tierra', '4 Otro', 'TOTAL']

IV4_censo = HOGAR_H06
IV4_censo.columns = ['radio', '1 Membrana / cubierta asfáltica', '2 Baldosa / losa sin cubierta', '3 Pizarra / teja', 
                     '4 Chapa de metal sin cubierta', '5 Chapa de fibrocemento / plástico', '6 Chapa de cartón', 
                     '7 Caña / tabla / paja con barro / paja sola', '9 N/S. Depto en propiedad horizontal', 'TOTAL']

IV5_censo = HOGAR_H07
IV6_censo = HOGAR_H08
IV7_censo = HOGAR_H09
IV7_censo['4 Otra fuente'] = HOGAR_H09['4 Pozo'] + HOGAR_H09['5 Transporte por cisterna'] + \
 HOGAR_H09[u'6 Agua de lluvia, río, canal, arroyo o acequia']
IV7_censo = IV7_censo[[u'radio', u'1 Red pública', u'2 Perforación con bomba a motor',
                      u'3 Perforación con bomba manual', u'4 Otra fuente', u'TOTAL']]

IV8_censo = HOGAR_H10
IV10_censo = HOGAR_H11
IV11_censo = HOGAR_H12
II9_censo = HOGAR_H13

II8_censo = HOGAR_H14
II8_censo['2 Gas de tubo / garrafa'] = HOGAR_H14['3 Gas en tubo'] + HOGAR_H14['4 Gas en garrafa']
II8_censo[u'3 Kerosene / leña/ carbón'] = HOGAR_H14[u'6 Leña o carbón']
II8_censo['4 Otro'] = HOGAR_H14[u'2 Gas a granel (zeppelin)'] + HOGAR_H14[u'5 Electricidad'] + HOGAR_H14[u'7 Otro']

II8_censo = II8_censo[['radio', '1 Gas de red', '2 Gas de tubo / garrafa', u'3 Kerosene / leña/ carbón', '4 Otro', 'TOTAL']]

II7_censo = HOGAR_PROP

IX_TOT_censo = HOGAR_TOTPERS

## Intento Max Ent

#### Margins

In [6]:
# From whole country data, we can only have margins. Same for a given district or precint.

tot_viviendas = 12171675.
IV4_censo.iloc[:, 1:].sum()/tot_viviendas

1 Membrana / cubierta asfáltica                0.179288
2 Baldosa / losa sin cubierta                  0.305800
3 Pizarra / teja                               0.095610
4 Chapa de metal sin cubierta                  0.361516
5 Chapa de fibrocemento / plástico             0.024129
6 Chapa de cartón                              0.008900
7 Caña / tabla / paja con barro / paja sola    0.014952
9 N/S. Depto en propiedad horizontal           0.009804
TOTAL                                          1.000000
dtype: float64

In [7]:
IV3_censo.iloc[:, 1:].sum()/tot_viviendas

1 Mosaico / baldosa / madera / cerámica / alfombra    0.765829
2 Cemento / ladrillo fijo                             0.202845
3 Ladrillo suelto / tierra                            0.025842
4 Otro                                                0.005484
TOTAL                                                 1.000000
dtype: float64

In [8]:
IV7_censo.iloc[:, 1:].sum()/tot_viviendas

1 Red pública                      0.839292
2 Perforación con bomba a motor    0.116497
3 Perforación con bomba manual     0.007038
4 Otra fuente                      0.037174
TOTAL                              1.000000
dtype: float64

#### Prior probabilities (matrix entries) from survey info

In [9]:
qjkl = df.groupby(['IV4', 'IV3', 'IV7'])[['CODUSU']].nunique() # how many IDs with each combination
qjkl /= qjkl.sum()
qjkl = qjkl.unstack().unstack().fillna(0)

In [10]:
qjkl #This is like a 8 x 4 x 4 cube

CODUSU                                                              \
IV7         1                                       2                       
IV3         1         2         3         4         1         2         3   
IV4                                                                         
1    0.261151  0.012960  0.000843  0.000695  0.005977  0.001006  0.000030   
2    0.199962  0.022206  0.000429  0.000429  0.009735  0.001923  0.000000   
3    0.063986  0.001894  0.000044  0.000104  0.005918  0.000237  0.000030   
4    0.235794  0.059237  0.001583  0.000429  0.011480  0.005933  0.000133   
5    0.009350  0.002264  0.000207  0.000192  0.000784  0.000325  0.000015   
6    0.000473  0.000592  0.000296  0.000044  0.000059  0.000178  0.000044   
7    0.003181  0.002915  0.000518  0.000044  0.000044  0.000074  0.000015   
9    0.070540  0.000473  0.000015  0.000399  0.000399  0.000044  0.000000   

                                                                      \
IV7                   3                                  4             
IV3         4         1         2         3    4         1         2   
IV4                                                                    
1    0.000015  0.000030  0.000030  0.000000  0.0  0.000104  0.000000   
2    0.000000  0.000089  0.000059  0.000000  0.0  0.000118  0.000133   
3    0.000000  0.000089  0.000030  0.000000  0.0  0.000089  0.000000   
4    0.000030  0.000118  0.000133  0.000015  0.0  0.000636  0.000473   
5    0.000000  0.000000  0.000000  0.000015  0.0  0.000000  0.000030   
6    0.000000  0.000000  0.000000  0.000000  0.0  0.000015  0.000000   
7    0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000015   
9    0.000000  0.000015  0.000000  0.000000  0.0  0.000000  0.000000   

                         
IV7                      
IV3         3         4  
IV4                      
1    0.000000  0.000000  
2    0.000015  0.000000  
3    0.000000  0.000015  
4    0.000015  0.000015  
5    0.000030  0.000000  
6    0.000000  0.000000  
7    0.000000  0.000000  
9    0.000000  0.000000

In [11]:
## Model?
# It would be a max ent prolem minimizing distance between pjkl and priors qjkl, 
# using lagrange multipliers with the conditions that (hyper-) row and column sums match the margins.

In [12]:
# import scipy.maxentropy.conditionalmodel as cm

In [13]:
# def f_1(x):
#     return x

# F = [f_1]
# samplespace = np.ones((8, 4, 4))
# K = qjkl.values.reshape(8, 4, 4)
# cm(F, samplespace, K)